In [32]:
import pandas as pd

df = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")

#remove all rows where the date column is empty
df = df[df["date"].notna()]

#date format
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dropna(subset="date", inplace=True)
df['date'] = df['date'].dt.date
df['date'] = pd.to_datetime(df['date'])

def replace_earliest_date(df):
    # Convert 'date' to datetime format and filter the data for January 2010
    df['date'] = pd.to_datetime(df['date'])
    jan_2010_filter = df['date'].dt.strftime('%Y-%m') == '2010-01'
    jan_2010_data = df[jan_2010_filter]

    # Find the earliest date for each stock
    earliest_dates = jan_2010_data.groupby('stock_RIC')['date'].min()

    # Replace the earliest dates with 2010-01-01
    df.loc[df.set_index(['stock_RIC', 'date']).index.isin(earliest_dates.reset_index().set_index(['stock_RIC', 'date']).index), 'date'] = pd.Timestamp('2009-12-30')

    return df

df = replace_earliest_date(df)

# def append_earliest_row(data, start_date, end_date, new_date):
#     # Convert the 'date' column to datetime if it's not already
#     data['date'] = pd.to_datetime(data['date'])

#     data = data[data['date'] != pd.to_datetime(new_date)]
    
#     # Filter rows that have dates within July 2019
#     filtered_data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]
    
#     # Group by 'stock_RIC' and get the earliest date for each group
#     earliest_rows = filtered_data.groupby('stock_RIC').apply(lambda x: x.nsmallest(1, 'date')).reset_index(drop=True)
    
#     # Change the 'date' to '2019-06-30'
#     earliest_rows['date'] = pd.Timestamp(new_date)
    
#     # Append these modified rows back to the original data frame
#     new_data = pd.concat([data, earliest_rows], ignore_index=True)
    
#     return new_data

# df = append_earliest_row(df, '2019-07-01', '2019-07-31', '2019-06-30')
# df = append_earliest_row(df, '2014-10-01', '2014-10-30', '2014-09-30')

def get_last_observation_next_month(df, output_file):

    # Ensure the date column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Sort the DataFrame by stock and date to ensure the last observation can be found
    df.sort_values(by=['stock_RIC', 'date'], ascending=[True, False], inplace=True)

    # Group by stock and year-month, then take the last observation
    df['year_month'] = df['date'].dt.to_period('M')
    last_observation_df = df.groupby(['stock_RIC', 'year_month']).agg('last').reset_index()

    # Change the date to the first day of the next month
    last_observation_df['date'] = (last_observation_df['date'] + pd.offsets.MonthEnd(1)) + pd.Timedelta(days=1)

    # Drop the year_month column as it's no longer needed
    last_observation_df.drop('year_month', axis=1, inplace=True)    

    last_observation_df = last_observation_df.drop('index', axis=1)
    # Save the resulting DataFrame to a new CSV file
    last_observation_df.to_csv(output_file, index=False)
    print(f"Processed data has been saved to {output_file}")

# Specify the path to your CSV file and the path for the output file
#input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/m_stock_level_data.csv'

# Call the function to process the file 
get_last_observation_next_month(df, output_file)

Processed data has been saved to /Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/m_stock_level_data.csv


,index,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price,year_month
1185671,1185671,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.133779,-6.675567e-02,938065.0,1.404420e+07,7.302660e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
1185672,1185672,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.066845,-6.675567e-02,0.0,1.404420e+07,7.305600e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
1185673,1185673,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.066845,-5.705210e-10,0.0,1.404420e+07,7.298249e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
1185674,1185674,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.133779,6.684492e-02,0.0,1.404420e+07,7.264478e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
1185675,1185675,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,-0.133422,1.337793e-01,0.0,1.404420e+07,7.266316e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,15252,ZURN.S,2010-01-08,157.583688,0.649632,2.604857,5.110810e+00,770634.0,1.213405e+08,2.323664e+10,NaN,1.148270,157.448074,157.583688,2010-01
15251,15251,ZURN.S,2010-01-07,156.053008,-1.366937,1.942605,3.962179e+00,835262.0,1.305151e+08,2.301093e+10,NaN,1.129708,155.985423,156.053008,2010-01
15250,15250,ZURN.S,2010-01-06,158.087559,-0.382979,3.355408,4.369148e+00,659170.0,1.040779e+08,2.331094e+10,NaN,1.151941,158.087559,158.155089,2010-01
15249,15249,ZURN.S,2010-01-05,158.256072,2.173913,3.115401,4.770397e+00,974130.0,1.533522e+08,2.333579e+10,NaN,1.149657,158.256072,158.323415,2010-01


### aligning dates

In [8]:
import pandas as pd

# Step 1: Load the CSV file
file_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv'
df = pd.read_csv(file_path)

# Convert all dates to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dropna(subset="date", inplace=True)

df = df.drop('test_x', axis=1)
display(df)

# Save the modified DataFrame
new_file_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data_new.csv'
df.to_csv(new_file_path, index=False)


/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_3936/3051638257.py:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price
0,1U1.DE,2009-12-30,5.050000,1.814516,-0.980392,-4.716981,0.0,7.655200e+05,2.686045e+08,78255000.0,2.068073,4.930000,4.980000
1,1U1.DE,2010-01-04,4.960000,-1.782178,-2.745098,-6.238185,293192.0,1.450690e+06,2.638175e+08,78255000.0,2.031216,4.940000,4.949000
2,1U1.DE,2010-01-05,4.920000,-0.806452,-0.806452,-6.994329,427069.0,2.086250e+06,2.616900e+08,78255000.0,2.014835,4.900000,4.920000
3,1U1.DE,2010-01-06,4.780000,-2.845528,-5.346535,-9.640832,355622.0,1.710380e+06,2.542435e+08,78255000.0,1.957502,4.780000,4.790000
4,1U1.DE,2010-01-07,4.640000,-2.928870,-8.118812,-11.619048,933668.0,4.349370e+06,2.467970e+08,78255000.0,1.900170,4.672000,4.688000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3170561,ZURN.S,2023-12-21,467.319907,-0.542005,-1.166966,1.685523,194600.0,9.099058e+07,6.839496e+10,NaN,3.183095,467.213794,467.319907
3170562,ZURN.S,2023-12-22,467.883798,0.068120,-0.854893,2.108434,157934.0,7.391228e+07,6.847749e+10,NaN,3.187497,467.883798,467.989966
3170563,ZURN.S,2023-12-27,468.655209,-0.476515,-0.948509,0.642497,142786.0,6.696645e+07,6.859039e+10,NaN,3.219729,468.548357,468.655209
3170564,ZURN.S,2023-12-28,466.683790,-0.638395,-1.044505,0.206944,161069.0,7.540347e+07,6.830186e+10,NaN,3.193492,466.683790,466.790876


In [26]:
import pandas as pd

# Step 1: Load the CSV file
file_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv'
df = pd.read_csv(file_path)

# Convert all dates to datetime format
df['stock_RIC'] = pd.to_datetime(df["stock_RIC"], errors='coerce')
df.dropna(subset="stock_RIC", inplace=True)


headers = ["stock_RIC", "date", "price", "return1D", "return1Wk", "return1Mo", "volume", "turnover", "market_cap", "gross_profit_new", "gross_profit_old", "price_to_BV", "bid_price", "ask_price"]
df.columns = headers
df = df.drop('ask_price', axis=1)
df.columns = ["stock_RIC", "date", "price", "return1D", "return1Wk", "return1Mo", "volume", "turnover", "market_cap", "gross_profit", "price_to_BV", "bid_price", "ask_price"]
display(df)


df['date'] = df['date'].dt.date
df['date'] = pd.to_datetime(df['date'])

# Save the modified DataFrame
new_file_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data_anomalies.csv'
df.to_csv(new_file_path, index=False)

/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_3936/1610150548.py:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_3936/1610150548.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['stock_RIC'] = pd.to_datetime(df["stock_RIC"], errors='coerce')


,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price
3170566,0MW4EUR.xbo^K15,2014-09-01 00:00:00+00:00,11.65,0.171969,0.691443,4.672058,2447859.0,2.845850e+07,5.671869e+09,2.668610e+09,NaN,11.480000,11.960000
3170567,0MW4EUR.xbo^K15,2014-09-02 00:00:00+00:00,11.81,1.373391,-0.169062,6.109614,3596408.0,4.269260e+07,5.749213e+09,2.668610e+09,NaN,11.710000,11.980000
3170568,0MW4EUR.xbo^K15,2014-09-03 00:00:00+00:00,12.04,1.947502,2.207131,8.176101,3918912.0,4.726140e+07,5.858756e+09,2.668610e+09,NaN,11.950000,12.130000
3170569,0MW4EUR.xbo^K15,2014-09-04 00:00:00+00:00,12.18,1.162791,4.909561,9.237668,3040034.0,3.667660e+07,5.928055e+09,2.668610e+09,NaN,12.070000,12.470000
3170570,0MW4EUR.xbo^K15,2014-09-05 00:00:00+00:00,12.2,0.164204,4.901118,10.909091,2575118.0,3.147450e+07,5.936712e+09,2.668610e+09,NaN,11.870000,12.240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213194,ZURN.S,2019-06-24 00:00:00+00:00,307.3688,0.088209,0.561300,3.402187,494401.0,1.518101e+08,4.598484e+10,NaN,1.708698,307.368800,307.459097
3213195,ZURN.S,2019-06-25 00:00:00+00:00,306.860735,-0.088132,-0.293169,3.311057,343044.0,1.051432e+08,4.590883e+10,NaN,1.701240,306.590055,306.860735
3213196,ZURN.S,2019-06-26 00:00:00+00:00,305.902215,0.000000,0.058841,3.311057,399112.0,1.222628e+08,4.576543e+10,NaN,1.696369,305.902215,305.992160
3213197,ZURN.S,2019-06-27 00:00:00+00:00,305.804163,-0.205822,0.029472,2.630783,347293.0,1.061349e+08,4.575076e+10,NaN,1.695825,305.714061,305.804163


### merge "anomalies" with stock_level_data.csv

In [30]:
anomalies_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data_anomalies.csv'
df_a = pd.read_csv(anomalies_path)

file_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data_new.csv'
df_stock_level_data_new = pd.read_csv(file_path)

display(df_a)
display(df_stock_level_data_new)

df_appended = pd.concat([df_stock_level_data_new, df_a], ignore_index=True)
display(df_appended)
df_appended.to_csv(file_path)

,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price
0,0MW4EUR.xbo^K15,2014-09-01,11.650000,0.171969,0.691443,4.672058,2447859.0,2.845850e+07,5.671869e+09,2.668610e+09,NaN,11.480000,11.960000
1,0MW4EUR.xbo^K15,2014-09-02,11.810000,1.373391,-0.169062,6.109614,3596408.0,4.269260e+07,5.749213e+09,2.668610e+09,NaN,11.710000,11.980000
2,0MW4EUR.xbo^K15,2014-09-03,12.040000,1.947502,2.207131,8.176101,3918912.0,4.726140e+07,5.858756e+09,2.668610e+09,NaN,11.950000,12.130000
3,0MW4EUR.xbo^K15,2014-09-04,12.180000,1.162791,4.909561,9.237668,3040034.0,3.667660e+07,5.928055e+09,2.668610e+09,NaN,12.070000,12.470000
4,0MW4EUR.xbo^K15,2014-09-05,12.200000,0.164204,4.901118,10.909091,2575118.0,3.147450e+07,5.936712e+09,2.668610e+09,NaN,11.870000,12.240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38002,ZURN.S,2019-06-24,307.368800,0.088209,0.561300,3.402187,494401.0,1.518101e+08,4.598484e+10,NaN,1.708698,307.368800,307.459097
38003,ZURN.S,2019-06-25,306.860735,-0.088132,-0.293169,3.311057,343044.0,1.051432e+08,4.590883e+10,NaN,1.701240,306.590055,306.860735
38004,ZURN.S,2019-06-26,305.902215,0.000000,0.058841,3.311057,399112.0,1.222628e+08,4.576543e+10,NaN,1.696369,305.902215,305.992160
38005,ZURN.S,2019-06-27,305.804163,-0.205822,0.029472,2.630783,347293.0,1.061349e+08,4.575076e+10,NaN,1.695825,305.714061,305.804163


,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price
0,1U1.DE,2009-12-30,5.050000,1.814516,-0.980392,-4.716981,0.0,7.655200e+05,2.686045e+08,78255000.0,2.068073,4.930000,4.980000
1,1U1.DE,2010-01-04,4.960000,-1.782178,-2.745098,-6.238185,293192.0,1.450690e+06,2.638175e+08,78255000.0,2.031216,4.940000,4.949000
2,1U1.DE,2010-01-05,4.920000,-0.806452,-0.806452,-6.994329,427069.0,2.086250e+06,2.616900e+08,78255000.0,2.014835,4.900000,4.920000
3,1U1.DE,2010-01-06,4.780000,-2.845528,-5.346535,-9.640832,355622.0,1.710380e+06,2.542435e+08,78255000.0,1.957502,4.780000,4.790000
4,1U1.DE,2010-01-07,4.640000,-2.928870,-8.118812,-11.619048,933668.0,4.349370e+06,2.467970e+08,78255000.0,1.900170,4.672000,4.688000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3170561,ZURN.S,2023-12-21,467.319907,-0.542005,-1.166966,1.685523,194600.0,9.099058e+07,6.839496e+10,NaN,3.183095,467.213794,467.319907
3170562,ZURN.S,2023-12-22,467.883798,0.068120,-0.854893,2.108434,157934.0,7.391228e+07,6.847749e+10,NaN,3.187497,467.883798,467.989966
3170563,ZURN.S,2023-12-27,468.655209,-0.476515,-0.948509,0.642497,142786.0,6.696645e+07,6.859039e+10,NaN,3.219729,468.548357,468.655209
3170564,ZURN.S,2023-12-28,466.683790,-0.638395,-1.044505,0.206944,161069.0,7.540347e+07,6.830186e+10,NaN,3.193492,466.683790,466.790876


,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price
0,1U1.DE,2009-12-30,5.050000,1.814516,-0.980392,-4.716981,0.0,7.655200e+05,2.686045e+08,78255000.0,2.068073,4.930000,4.980000
1,1U1.DE,2010-01-04,4.960000,-1.782178,-2.745098,-6.238185,293192.0,1.450690e+06,2.638175e+08,78255000.0,2.031216,4.940000,4.949000
2,1U1.DE,2010-01-05,4.920000,-0.806452,-0.806452,-6.994329,427069.0,2.086250e+06,2.616900e+08,78255000.0,2.014835,4.900000,4.920000
3,1U1.DE,2010-01-06,4.780000,-2.845528,-5.346535,-9.640832,355622.0,1.710380e+06,2.542435e+08,78255000.0,1.957502,4.780000,4.790000
4,1U1.DE,2010-01-07,4.640000,-2.928870,-8.118812,-11.619048,933668.0,4.349370e+06,2.467970e+08,78255000.0,1.900170,4.672000,4.688000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3208568,ZURN.S,2019-06-24,307.368800,0.088209,0.561300,3.402187,494401.0,1.518101e+08,4.598484e+10,NaN,1.708698,307.368800,307.459097
3208569,ZURN.S,2019-06-25,306.860735,-0.088132,-0.293169,3.311057,343044.0,1.051432e+08,4.590883e+10,NaN,1.701240,306.590055,306.860735
3208570,ZURN.S,2019-06-26,305.902215,0.000000,0.058841,3.311057,399112.0,1.222628e+08,4.576543e+10,NaN,1.696369,305.902215,305.992160
3208571,ZURN.S,2019-06-27,305.804163,-0.205822,0.029472,2.630783,347293.0,1.061349e+08,4.575076e+10,NaN,1.695825,305.714061,305.804163


##### ANOMALIE with dates

In [1]:
import pandas as pd

df = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].dt.date
df.dropna(subset=['date'], inplace=True)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

#### drop all dates in 2014-09
df = df[(df['date'].dt.year != 2014) | (df['date'].dt.month != 9)] #2014-10-01 anomalie
df = df[(df['date'].dt.year != 2019) | (df['date'].dt.month != 6)] #2019-07-01 anomalie 


####### append new data by running script

print(df.dtypes)
#df.to_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")

stock_RIC               object
date            datetime64[ns]
price                  float64
return1D               float64
return1Wk              float64
return1Mo              float64
volume                 float64
turnover               float64
market_cap             float64
gross_profit           float64
price_to_BV            float64
bid_price              float64
ask_price              float64
dtype: object
